In [5]:
import minsearch
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

### Index Documents

In [8]:
index = minsearch.Index(
        text_fields=["question", "text", "section"],  #use for performant search, look for text that can help answer the question
        keyword_fields=["course"]  #which documents to look in
        )

In [9]:
index.fit(documents)

In [10]:
q = "the course has already started, can i still enroll?"

In [11]:
boost = {"question": 3.0, "section": 0.5} # if a section of document has relevant terms, attirbute more importance to those sections. For example, question field is 3x more important than text field

results = index.search(
    query = q,
    boost_dict=boost,
    num_results = 5
    )

In [12]:
results_de_zoom = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results = 5
)

results_de_zoom

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

### Use GPT-4o model for generic response

In [33]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

In [ ]:
#generic response with 0 context
response.choices[0].message.content

"Whether you can still enroll in a course after it has started typically depends on the institution or platform offering the course. Here are some factors to consider:\n\n1. **Institution Policies**: Check with the institution or program guidelines. Some might allow late enrollment with or without a penalty, while others might not.\n\n2. **Course Format**: Online courses may offer more flexibility in terms of start dates and late enrollment compared to traditional in-person classes.\n\n3. **Instructor Approval**: Sometimes, you may be able to join late with the approval of the instructor, especially if you can demonstrate that you'll be able to catch up on missed material.\n\n4. **Duration and Content**: For courses with a short duration or those that cover a lot of material in a short period, late enrollment might not be an option due to the difficulty of catching up.\n\n5. **Availability of Resources**: Ensure that all necessary resources such as course materials, lecture recordings,

### Use GPT-4o model for specific response

In [17]:
context = ""
for doc in results: 
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [18]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question. If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {q} 

CONTEXT: {context}

""".strip()

In [21]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt_template}]
)

response.choices[0].message.content

'Yes, you can still enroll. You won’t be able to submit some of the homeworks, but you can still take part in the course. To get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline. So, if you join the course late and manage to work on two projects, you will still be eligible for a certificate.'

### Modularize the code

In [13]:
def search(query):
    boost = {"question": 3.0, "section": 0.5} 

    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results = 5
        )
    
    return results

In [14]:
def build_prompt(query, search_results):
    
    context = ""
    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    
    prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.

QUESTION: {query} 

CONTEXT: {context}

""".strip()
    
    return prompt

In [15]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

    return response.choices[0].message.content 

In [16]:
query = "how do i run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query=query, search_results=search_results )
    return llm(prompt=prompt)

In [42]:
rag(query)

'To run Kafka, follow these instructions based on your setup:\n\n1. **Java Kafka**: Navigate to your project directory and run the following command in the terminal:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **Python Kafka**:\n   - If you\'re using a `producer.py` file and encountering a "module not found" error, create a virtual environment and install the required packages within it:\n     ```bash\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n     Activate the virtual environment whenever you need to run the Python files.\n\n   - Ensure all your Docker images are up and running before executing the scripts. Use the command:\n     ```bash\n     docker compose up -d\n     ```\n     to start all necessary containers.\n\n3. **Troubleshooting Kafka Broker Connection Error**:\n   - If you receive a `kafka.errors.NoBrokersAvailable` error, it indicates

In [43]:
rag('the course has already started, can i still enroll?')

"Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks even if you haven't registered. However, please be aware that there will be deadlines for turning in the final projects, so make sure not to leave everything for the last minute."

### Using Elasticsearch

In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch('http://localhost:9200')

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
from tqdm.auto import tqdm

/workspaces/llms/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc) # create a record in the index for each document

100%|██████████| 948/948 [00:04<00:00, 219.72it/s]


In [20]:
query = 'I just discovered the course. Can I still join it?'

In [29]:
# querying the data in the index

def elastic_search(query):

    search_query = {
        "size": 5, #how many index records to return
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #boost is determined by exponent ^ 
                        "type": "best_fields"
                    }
                },
                "filter": {   # basically a where caluse 
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs



In [31]:
def rag_elastic(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query=query, search_results=search_results )
    return llm(prompt=prompt)

In [35]:
rag_elastic(query=query)

'Yes, you can still join the course even after the start date. You are eligible to submit the homeworks without formally registering, but be mindful of the deadlines for turning in the final projects.'